In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/covid19-in-india/HospitalBedsIndia.csv
/kaggle/input/covid19-in-india/ICMRTestingLabs.csv
/kaggle/input/covid19-in-india/StatewiseTestingDetails.csv
/kaggle/input/covid19-in-india/IndividualDetails.csv
/kaggle/input/covid19-in-india/population_india_census2011.csv
/kaggle/input/covid19-in-india/AgeGroupDetails.csv
/kaggle/input/covid19-in-india/covid_19_india.csv


In [3]:
!pip install bar_chart_race

     |████████████████████████████████| 156 kB 2.7 MB/s eta 0:00:01


In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import bar_chart_race as bcr
from IPython.display import HTML
import warnings
warnings.filterwarnings("ignore")
from IPython.display import Video

In [5]:
covid_df = pd.read_csv('../input/covid19-in-india/covid_19_india.csv', index_col=False)
covid_df.head()

,Sno,Date,Time,State/UnionTerritory,ConfirmedIndianNational,ConfirmedForeignNational,Cured,Deaths,Confirmed
0,1,30/01/20,6:00 PM,Kerala,1,0,0,0,1
1,2,31/01/20,6:00 PM,Kerala,1,0,0,0,1
2,3,01/02/20,6:00 PM,Kerala,2,0,0,0,2
3,4,02/02/20,6:00 PM,Kerala,3,0,0,0,3
4,5,03/02/20,6:00 PM,Kerala,3,0,0,0,3


In [7]:
covid_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2919 entries, 0 to 2918
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Sno                       2919 non-null   int64 
 1   Date                      2919 non-null   object
 2   Time                      2919 non-null   object
 3   State/UnionTerritory      2919 non-null   object
 4   ConfirmedIndianNational   2919 non-null   object
 5   ConfirmedForeignNational  2919 non-null   object
 6   Cured                     2919 non-null   int64 
 7   Deaths                    2919 non-null   int64 
 8   Confirmed                 2919 non-null   int64 
dtypes: int64(4), object(5)
memory usage: 205.4+ KB


In [8]:
# change datatype of date to a pandas datetime format
covid_df['Date'] = pd.to_datetime(covid_df['Date'], dayfirst=True)

#covid_data_complete["Date"] = covid_data_complete["Date"].apply(pd.to_datetime)

#drop columns other than total_cases
drop_cols = ['Sno', 'Time', 'ConfirmedIndianNational', 'ConfirmedForeignNational', 'Cured', 'Deaths']
covid_df.drop(covid_df[drop_cols],axis=1,inplace=True)


covid_df = covid_df[covid_df['State/UnionTerritory'] != 'Cases being reassigned to states']

#dropping data before Feb 29 2020 as only Kerala had 3 cases till then in a span of one month
covid_df = covid_df[covid_df['Date'] > pd.to_datetime(pd.DataFrame({'year': [2020],'month': [2],'day': [28]}))[0]]

covid_df.head()

,Date,State/UnionTerritory,Confirmed
30,2020-02-29,Kerala,3
31,2020-03-01,Kerala,3
32,2020-03-02,Telengana,1
33,2020-03-02,Kerala,3
34,2020-03-02,Delhi,1


In [10]:
covid_data = covid_df.copy() #make a copy for analysis
covid_data.columns = ['Date', 'States', 'Cases'] #rename columns
covid_data.head(10)

,Date,States,Cases
30,2020-02-29,Kerala,3
31,2020-03-01,Kerala,3
32,2020-03-02,Telengana,1
33,2020-03-02,Kerala,3
34,2020-03-02,Delhi,1
35,2020-03-03,Telengana,1
36,2020-03-03,Rajasthan,1
37,2020-03-03,Kerala,3
38,2020-03-03,Delhi,1
39,2020-03-04,Uttar Pradesh,6


In [11]:
total_states = covid_data['States'].nunique()
total_states

37

In [12]:
# set states and date as index and find cases
# transpose the dataframe to have countries as columns and dates as rows
covid_data_by_date = covid_data.set_index(['States','Date']).unstack()['Cases'].T.reset_index()

covid_data_by_date = covid_data_by_date.set_index('Date') #make date as index - desired by barchartrace

covid_data_by_date = covid_data_by_date.fillna(0) #fill na with 0

covid_data_by_date

States,Andaman and Nicobar Islands,Andhra Pradesh,Arunachal Pradesh,Assam,Bihar,Chandigarh,Chhattisgarh,Dadar Nagar Haveli,Daman & Diu,Delhi,...,Punjab,Rajasthan,Sikkim,Tamil Nadu,Telengana,Tripura,Unassigned,Uttar Pradesh,Uttarakhand,West Bengal
Date,,,,,,,,,,,,,,,,,,,,,
2020-02-29,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-03-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-03-02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2020-03-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2020-03-04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,15.0,0.0,0.0,1.0,0.0,0.0,6.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-06-07,33.0,4510.0,47.0,2397.0,4915.0,309.0,923.0,19.0,0.0,27654.0,...,2515.0,10331.0,7.0,30152.0,3496.0,747.0,0.0,9733.0,1303.0,7738.0
2020-06-08,33.0,4708.0,51.0,2565.0,5088.0,314.0,1073.0,20.0,0.0,27654.0,...,2608.0,10599.0,7.0,31667.0,3580.0,800.0,0.0,10536.0,1355.0,8187.0
2020-06-09,33.0,4851.0,51.0,2776.0,5202.0,317.0,1160.0,22.0,0.0,29943.0,...,2663.0,10763.0,7.0,33229.0,3650.0,838.0,0.0,10947.0,1411.0,8613.0


In [15]:
#make the mp4 file with the BarChartRace and save it

bcr.bar_chart_race(
    df=covid_data_by_date,
    filename='India_Covid19_BarChartRace.mp4',
    orientation='h',
    sort='desc',
    n_bars=10,
    fixed_order=False,
    fixed_max=False,
    steps_per_period=10,
    interpolate_period=False,
    label_bars=True,
    bar_size=.95,
    period_label={'x': .99, 'y': .25, 'ha': 'right', 'va': 'center'},
    period_fmt='%B %d, %Y',
    period_summary_func=lambda v, r: {'x': .99, 'y': .05,
                                      's': f'Total cases: {v.nlargest(total_states).sum():,.0f}',
                                      #'s': '',
                                      'ha': 'right', 'size': 10, 'family': 'Courier New'},
    perpendicular_bar_func='median',
    period_length=1000,
    figsize=(5, 3),
    dpi=500,
    cmap='dark24',
    title='COVID-19 cases in India',
    title_size=10,
    bar_label_size=10,
    tick_label_size=10,
    shared_fontdict={'color' : '.1'},
    scale='linear',
    writer=None,
    fig=None,
    bar_kwargs={'alpha': .7},
    filter_column_colors=True) 

Exception: You do not have ffmpeg installed on your machine. Download
                            ffmpeg from here: https://www.ffmpeg.org/download.html.
                            
                            Matplotlib's original error message below:

                            'MovieWriterRegistry' object is not an iterator
                            